- 参考 [天秀！GitHub 硬核项目：动漫生成器让照片秒变手绘日漫风！！！](https://mp.weixin.qq.com/s?__biz=MzAxOTcxNTIwNQ==&mid=310435176&idx=1&sn=9d3f5916ae5126c4e3233b26595e02cb&chksm=0cb6b8823bc13194bb38ce5eabe344e59a6881f1ae6a4ac0aa183874ec71c2586b73ce3c0f96#rd)  
以下下步骤中 [ ] 表示非必须

- **[查看分配到的GPU型号]**

In [0]:
!nvidia-smi

- **1. 安装运行环境**

In [0]:
!pip3 install tensorflow-gpu==1.13.1
!pip3 install opencv-python
!pip3 install tqdm
!pip3 install numpy
!pip3 install argparse

- **[检查是否安装成功]**

In [0]:
!pip3 show tensorflow-gpu
!pip3 show opencv-python
!pip3 show tqdm
!pip3 show numpy
!pip3 show argparse
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

- **2. 下载AnimeGAN源码**

In [0]:
%cd /content
!git clone https://github.com/TachibanaYoshino/AnimeGAN

- **3. 添加(or上传)并执行预训练模型下载脚本 download_staffs.sh**

In [0]:
%cd /content/AnimeGAN
!echo -e 'URL=https://github.com/TachibanaYoshino/AnimeGAN/releases/download/Haoyao-style_V1.0/Haoyao-style.zip\nZIP_FILE=./checkpoint/Haoyao-style.zip\nTARGET_DIR=./checkpoint/saved_model\n\nmkdir -p ./checkpoint\nwget -N $URL -O $ZIP_FILE\nmkdir -p $TARGET_DIR\nunzip $ZIP_FILE -d $TARGET_DIR\nrm $ZIP_FILE\n\nDatesetURL=https://github.com/TachibanaYoshino/AnimeGAN/releases/download/dataset-1/dataset.zip\nZIP_FILE=./dataset.zip\nTARGET_DIR=./dataset\n\nrm -rf dataset\nwget -N $DatesetURL -O $ZIP_FILE\nunzip $ZIP_FILE -d $TARGET_DIR\nrm $ZIP_FILE\n\nVGG_FILE=./vgg19_weight/vgg19.npy\nwget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate \0047https://docs.google.com/uc?export=download&id=1U5HCRpZWAbDVLipNoF8t0ZHpwCRX7kdF\0047 -O- | sed -rn \0047s/.*confirm=([0-9A-Za-z_]+).*/\01341\0134n/p\0047)&id=1U5HCRpZWAbDVLipNoF8t0ZHpwCRX7kdF" -O $VGG_FILE && rm -rf /tmp/cookies.txt' > download_staffs.sh
!bash download_staffs.sh

- **4. [训练模型(非常耗时, 可以跳过这一步)]**

In [0]:
!python main.py --phase train --dataset Hayao --epoch 101 --init_epoch 1

- **5. 添加(or上传)视频处理脚本 video.py**

In [0]:
!echo -e 'import\0040argparse\nimport\0040cv2\nimport\0040os\nimport\0040re\nimport\0040shutil\n\nfg_video_path\0040=\0040\0047/content/AnimeGAN/video/input_video.mp4\0047\nEXTRACT_FREQUENCY\0040=\00401\n\ndef\0040extract(video_path,\0040image_path,\0040index=EXTRACT_FREQUENCY):\n\0040\0040\0040\0040try:\n\0040\0040\0040\0040\0040\0040\0040\0040shutil.rmtree(image_path)\n\0040\0040\0040\0040except\0040OSError:\n\0040\0040\0040\0040\0040\0040\0040\0040pass\n\0040\0040\0040\0040os.mkdir(image_path)\n\0040\0040\0040\0040video\0040=\0040cv2.VideoCapture()\n\0040\0040\0040\0040if\0040not\0040video.open(video_path):\n\0040\0040\0040\0040\0040\0040\0040\0040print("can\0040not\0040open\0040the\0040video")\n\0040\0040\0040\0040\0040\0040\0040\0040exit(1)\n\0040\0040\0040\0040count\0040=\00401\n\0040\0040\0040\0040while\0040True:\n\0040\0040\0040\0040\0040\0040\0040\0040_,\0040frame\0040=\0040video.read()\n\0040\0040\0040\0040\0040\0040\0040\0040if\0040frame\0040is\0040None:\n\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040break\n\0040\0040\0040\0040\0040\0040\0040\0040if\0040count\0040%\0040EXTRACT_FREQUENCY\0040==\00400:\n\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040save_path\0040=\0040"{}/{:>04d}.jpg".format(image_path,\0040index)\n\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040cv2.imwrite(save_path,\0040frame)\n\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040\0040index\0040+=\00401\n\0040\0040\0040\0040\0040\0040\0040\0040count\0040+=\00401\n\0040\0040\0040\0040video.release()\n\0040\0040\0040\0040print("Totally\0040save\0040{:d}\0040pics".format(index\0040-\00401))\n\ndef\0040is_frame(path):\n\0040\0040\0040\0040res\0040=\0040re.match(r\0047\0134d{4}\0134.jpg$\0047,\0040path)\n\0040\0040\0040\0040return\0040True\0040if\0040res\0040!=\0040None\0040else\0040False\n\ndef\0040filter_frame(array):\n\0040\0040\0040\0040res\0040=\0040[]\n\0040\0040\0040\0040for\0040item\0040in\0040filter(is_frame,\0040array):\n\0040\0040\0040\0040\0040\0040\0040\0040res.append(item)\n\0040\0040\0040\0040res.sort(reverse=False)\n\0040\0040\0040\0040return\0040res\n\ndef\0040combine(image_path,\0040output_path):\n\0040\0040\0040\0040cap\0040=\0040cv2.VideoCapture(fg_video_path)\n\0040\0040\0040\0040fgs\0040=\0040int(cap.get(cv2.CAP_PROP_FPS))\0040\n\0040\0040\0040\0040fgs\0040=\0040fgs\0040if\0040fgs\0040>\00400\0040else\004025\n\0040\0040\0040\0040pictrue_in_filelist\0040=\0040filter_frame(os.listdir(image_path))\n\0040\0040\0040\0040print(pictrue_in_filelist)\n\0040\0040\0040\0040name\0040=\0040image_path\0040+\0040"/"\0040+\0040pictrue_in_filelist[0]\n\0040\0040\0040\0040img\0040=\0040cv2.imread(name)\n\0040\0040\0040\0040h,\0040w,\0040c\0040=\0040img.shape\n\0040\0040\0040\0040size\0040=\0040(w,\0040h)\n\0040\0040\0040\0040print(f\0047size:\0040{size},\0040fgs:\0040{fgs}\0047)\n\n\0040\0040\0040\0040fourcc\0040=\0040cv2.VideoWriter_fourcc(*\0047XVID\0047)\n\0040\0040\0040\0040out_video\0040=\0040output_path\0040+\0040\0047.mp4\0047\n\0040\0040\0040\0040video_writer\0040=\0040cv2.VideoWriter(out_video,\0040fourcc,\0040fgs,\0040size)\n\n\0040\0040\0040\0040for\0040i\0040in\0040range(len(pictrue_in_filelist)):\n\0040\0040\0040\0040\0040\0040\0040\0040pictrue_in_filename\0040=\0040image_path\0040+\0040"/"\0040+\0040pictrue_in_filelist[i]\n\0040\0040\0040\0040\0040\0040\0040\0040img12\0040=\0040cv2.imread(pictrue_in_filename)\n\0040\0040\0040\0040\0040\0040\0040\0040video_writer.write(img12)\n\0040\0040\0040\0040video_writer.release()\n\0040\0040\0040\0040#print("删除合成的图片数据集")\n\0040\0040\0040\0040#shutil.rmtree(fg_in_bg)\n\0040\0040\0040\0040return\0040out_video\n\ndef\0040parse_args():\n\0040\0040\0040\0040desc\0040=\0040"video\0040util"\n\0040\0040\0040\0040parser\0040=\0040argparse.ArgumentParser(description=desc)\n\0040\0040\0040\0040parser.add_argument(\0047--type\0047,\0040type=str,\0040default=\0047extract\0047,\0040help=\0047specify\0040which\0040action\0040to\0040take\0047)\n\0040\0040\0040\0040parser.add_argument(\0047--video_path\0047,\0040type=str,\0040default=\0047/content/AnimeGAN/video/input_video.mp4\0047,\0040help=\0047input\0040video\0040path\0047)\n\0040\0040\0040\0040parser.add_argument(\0047--image_path\0047,\0040type=str,\0040default=\0047/content/AnimeGAN/video/input_video\0047,\0040help=\0047output\0040images\0040path\0047)\n\0040\0040\0040\0040parser.add_argument(\0047--output_path\0047,\0040type=str,\0040default=\0047/content/AnimeGAN/video/output_video.mp4\0047,\0040help=\0047output\0040video\0040path\0047)\n\0040\0040\0040\0040"""checking\0040arguments"""\n\0040\0040\0040\0040return\0040parser.parse_args()\n\nif\0040__name__\0040==\0040"__main__":\n\0040\0040\0040\0040args\0040=\0040parse_args()\n\0040\0040\0040\0040if\0040args.type\0040==\0040\0047extract\0047:\n\0040\0040\0040\0040\0040\0040\0040\0040extract(args.video_path,\0040args.image_path)\n\0040\0040\0040\0040\0040\0040\0040\0040print(f\0047extract\0040video\0040{args.video_path}\0040into\0040images\0040{args.image_path}\0047)\n\0040\0040\0040\0040elif\0040args.type\0040==\0040\0047combine\0047:\n\0040\0040\0040\0040\0040\0040\0040\0040combine(args.image_path,\0040args.output_path)\n\0040\0040\0040\0040\0040\0040\0040\0040print(f\0047combine\0040images\0040from\0040{args.image_path}\0040into\0040{args.output_path}\0047)\n\0040\0040\0040\0040else:\n\0040\0040\0040\0040\0040\0040\0040\0040print(f\0047Error:\0040you\0040must\0040specify\0040the\0040argument\0040of\0040type,\0040extract\0040or\0040combine\0047)' > video.py

- **6. 上传视频**

In [0]:
%mkdir video
print('Upload video in this dir: /content/AnimeGAN/video')

- **7. 视频分解为序列帧图片**

In [0]:
!python video.py --type extract --video_path /content/AnimeGAN/video/input_video.mp4 --image_path /content/AnimeGAN/video/input_video

- **8. 序列帧图片动漫化**

In [0]:
!python test.py --checkpoint_dir checkpoint/saved_model --test_dir video/input_video --style_name H

- **9. 序列帧合成视频**

In [0]:
!python video.py --type combine --image_path /content/AnimeGAN/results/H --output_path /content/AnimeGAN/video/output_video.mp4